# Лекция 1. Задание 2

Используя метод конечного объема, построить разностную схему для уравнения индукции в цилиндрической и сферической системах координат.

$$ \frac{\partial \vec{B}}{\partial t} = \text{rot} (\vec{v} \times \vec{B}) $$

Для примера приведем компоненты уравнения индукции в цилиндрических координатах:

$$ \frac{\partial B_r}{\partial t} + \frac{1}{r} \frac{\partial}{\partial \varphi} (v_\varphi B_r - v_r B_\varphi) + \frac{\partial}{\partial z} (v_z B_r - v_r B_z) = 0 $$

$$ \frac{\partial B_\varphi}{\partial t} + \frac{\partial}{\partial r} (v_r B_\varphi - v_\varphi B_r) + \frac{\partial}{\partial z} (v_z B_\varphi - v_\varphi B_z) = 0 $$

$$ \frac{\partial B_z}{\partial t} + \frac{1}{r} \frac{\partial}{\partial r} [r (v_r B_z - v_z B_r)] + \frac{1}{r} \frac{\partial}{\partial \varphi} (v_\varphi B_z - v_z B_\varphi) = 0 $$

## Решение 

In [42]:
from functools import reduce

import sympy as sy
from sympy import Derivative
from sympy import Eq
from sympy import Function
from sympy import Integral
from sympy import Symbol

## Цилиндрические координаты

In [125]:
r = Symbol('r')
r_1 = Symbol('r_1')
r_2 = Symbol('r_2')
r_c = Symbol('r_c')
phi = Symbol('\\varphi')
phi_1 = Symbol('\\varphi_1')
phi_2 = Symbol('\\varphi_2')
z = Symbol('z')
z_1 = Symbol('z_1')
z_2 = Symbol('z_2')
t = Symbol('t')
t_n = Symbol('t^n')
t_np1 = Symbol('t^{n + 1}')
Delta_r = Symbol('\\Delta r')
Delta_phi = Symbol('\\Delta \\varphi')
Delta_z = Symbol('\\Delta z')
Delta_t = Symbol('\\Delta t')
Delta_V = Symbol('\\Delta V')
B_r = Function('B_r')(r, t)
B_phi = Function('B_\\varphi')(phi, t)
B_z = Function('B_z')(z, t)
B_c = Function('B_c')(t) # ?
v_r = Function('v_r')(r, t)
v_phi = Function('v_\\varphi')(phi, t)
v_z = Function('v_z')(z, t)
# v_c = Function('v_c')(t) # ?

### Сетка

In [20]:
r_c_ = Eq(r_c, (r_1 + r_2) / 2)
r_c_

Eq(r_c, r_1/2 + r_2/2)

In [21]:
Delta_V_ = Eq(Delta_V, r_c * Delta_r * Delta_phi * Delta_z)
Delta_V_

Eq(\Delta V, \Delta \varphi*\Delta r*\Delta z*r_c)

### Уравнения

In [22]:
eq_phi_ = Eq(Derivative(B_phi, t) + Derivative(v_r * B_phi - v_phi * B_r, r) + Derivative(v_z * B_phi - v_phi * B_z, z), 0)
eq_phi_

Eq(Derivative(B_\varphi(\varphi, t)*v_r(r, t) - B_r(r, t)*v_\varphi(\varphi, t), r) + Derivative(B_\varphi(\varphi, t)*v_z(z, t) - B_z(z, t)*v_\varphi(\varphi, t), z) + Derivative(B_\varphi(\varphi, t), t), 0)

In [23]:
eq_r_ = Eq(Derivative(B_r, t) + 1 / r * Derivative(v_phi * B_r - v_r * B_phi, phi) + Derivative(v_z * B_r - v_r * B_z, z), 0)
eq_r_

Eq(Derivative(B_r(r, t)*v_z(z, t) - B_z(z, t)*v_r(r, t), z) + Derivative(B_r(r, t), t) + Derivative(-B_\varphi(\varphi, t)*v_r(r, t) + B_r(r, t)*v_\varphi(\varphi, t), \varphi)/r, 0)

In [24]:
eq_z_ = Eq(Derivative(B_z, t) + 1 / r * Derivative(r * (v_r * B_z - v_z * B_r), r) + 1 / r * Derivative(v_phi * B_z - v_z * B_phi, phi), 0)
eq_z_

Eq(Derivative(B_z(z, t), t) + Derivative(r*(-B_r(r, t)*v_z(z, t) + B_z(z, t)*v_r(r, t)), r)/r + Derivative(-B_\varphi(\varphi, t)*v_z(z, t) + B_z(z, t)*v_\varphi(\varphi, t), \varphi)/r, 0)

### Умножение на общий знаменатель

In [90]:
common_denominator_multiplication = lambda expr, denominator: (
    (expr * denominator).simplify())

In [92]:
eq_r_left_ = common_denominator_multiplication(eq_r_.args[0], r)
eq_r_left_

r*(B_r(r, t)*Derivative(v_z(z, t), z) - v_r(r, t)*Derivative(B_z(z, t), z) + Derivative(B_r(r, t), t)) + B_r(r, t)*Derivative(v_\varphi(\varphi, t), \varphi) - v_r(r, t)*Derivative(B_\varphi(\varphi, t), \varphi)

In [93]:
eq_phi_left_ = common_denominator_multiplication(eq_phi_.args[0], 1)
eq_phi_left_

B_\varphi(\varphi, t)*Derivative(v_r(r, t), r) + B_\varphi(\varphi, t)*Derivative(v_z(z, t), z) - v_\varphi(\varphi, t)*Derivative(B_r(r, t), r) - v_\varphi(\varphi, t)*Derivative(B_z(z, t), z) + Derivative(B_\varphi(\varphi, t), t)

In [95]:
eq_z_left_ = common_denominator_multiplication(eq_z_.args[0], r)
eq_z_left_

r*(B_z(z, t)*Derivative(v_r(r, t), r) - v_z(z, t)*Derivative(B_r(r, t), r)) + r*Derivative(B_z(z, t), t) - B_r(r, t)*v_z(z, t) + B_z(z, t)*v_r(r, t) + B_z(z, t)*Derivative(v_\varphi(\varphi, t), \varphi) - v_z(z, t)*Derivative(B_\varphi(\varphi, t), \varphi)

### Интегрирование по пространству и времени

In [126]:
integral_z = lambda expr: Integral(expr, (z, z_1, z_2))
integral_phi = lambda expr: Integral(expr, (phi, phi_1, phi_2))
integral_r = lambda expr: Integral(expr, (r, r_1, r_2))
integral_t = lambda expr: Integral(expr, (t, t_n, t_np1))

integral_chain = lambda expr: reduce(
    lambda prev, next: next(prev),
    [expr, integral_z, integral_phi, integral_r, integral_t])

# zero_divergence = lambda expr: (expr.subs(B_r, B_c).subs(B_phi, B_c)
#                                      .subs(B_z, B_c).subs(v_r, v_c)
#                                      .subs(v_phi, v_c).subs(v_z, v_c))
zero_divergence = lambda expr: (expr.subs(B_r, B_c)
                                    .subs(B_phi, B_c)
                                    .subs(B_z, B_c))

def integral_to_delta(expr):
    integrand = expr.args[0]
    if not z in integrand.free_symbols:
        integrand *= Delta_z
    else:
        integrand = integral_z(integrand)
    if not phi in integrand.free_symbols:
        integrand *=  Delta_phi
    else:
        integrand = integral_phi(integrand)
    if not r in integrand.free_symbols:
        integrand *= Delta_r
    else:
        integrand = integral_r(integrand)
    if not t in integrand.free_symbols:
        integrand *= Delta_t
    else:
        integrand = integral_t(integrand)
    return integrand

In [127]:
eq_r_integral_left_ = integral_chain(eq_r_left_.expand()).expand()
eq_r_integral_left_

Integral(r*Derivative(B_r(r, t), t), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(B_r(r, t)*Derivative(v_\varphi(\varphi, t), \varphi), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(-v_r(r, t)*Derivative(B_\varphi(\varphi, t), \varphi), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(r*B_r(r, t)*Derivative(v_z(z, t), z), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(-r*v_r(r, t)*Derivative(B_z(z, t), z), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1}))

In [134]:
eq_r_zero_divergence_left_ = zero_divergence(eq_r_integral_left_)
eq_r_zero_divergence_left_

Integral(r*Derivative(B_c(t), t), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(B_c(t)*Derivative(v_\varphi(\varphi, t), \varphi), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(-v_r(r, t)*Derivative(B_c(t), \varphi), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(r*B_c(t)*Derivative(v_z(z, t), z), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(-r*v_r(r, t)*Derivative(B_c(t), z), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1}))

In [137]:
eq_r_delta_left_ = sum([integral_to_delta(arg) for arg in eq_r_zero_divergence_left_.args])
eq_r_delta_left_

Integral(\Delta r*Integral(\Delta z*B_c(t)*Derivative(v_\varphi(\varphi, t), \varphi), (\varphi, \varphi_1, \varphi_2)), (t, t^n, t^{n + 1})) + Integral(\Delta \varphi*Integral(r*B_c(t)*Derivative(v_z(z, t), z), (z, z_1, z_2)), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(\Delta \varphi*\Delta z*r*Derivative(B_c(t), t), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(-\Delta \varphi*\Delta z*v_r(r, t)*Derivative(B_c(t), \varphi), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(-\Delta \varphi*\Delta z*r*v_r(r, t)*Derivative(B_c(t), z), (r, r_1, r_2), (t, t^n, t^{n + 1}))

In [138]:
eq_r_delta_left_.simplify()

\Delta \varphi*\Delta z*(r_1**2*B_c(t^n)/2 - r_1**2*B_c(t^{n + 1})/2 - r_2**2*B_c(t^n)/2 + r_2**2*B_c(t^{n + 1})/2) + \Delta \varphi*(r_1 - r_2)*(r_1 + r_2)*(Integral(B_c(t)*v_z(z_1, t), (t, t^n, t^{n + 1})) + Integral(-B_c(t)*v_z(z_2, t), (t, t^n, t^{n + 1})))/2 + \Delta r*\Delta z*(-Integral(B_c(t)*v_\varphi(\varphi_1, t), (t, t^n, t^{n + 1})) - Integral(-B_c(t)*v_\varphi(\varphi_2, t), (t, t^n, t^{n + 1})))

In [63]:
integral_chain(eq_phi_left_).expand()

Integral(Derivative(B_\varphi(\varphi, t)*v_r(r, t) - B_r(r, t)*v_\varphi(\varphi, t), r), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(Derivative(B_\varphi(\varphi, t)*v_z(z, t) - B_z(z, t)*v_\varphi(\varphi, t), z), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(Derivative(B_\varphi(\varphi, t), t), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1}))

In [64]:
integral_chain(eq_z_left_).expand()

Integral(r*Derivative(B_z(z, t), t), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(-B_r(r, t)*v_z(z, t), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(B_z(z, t)*v_r(r, t), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(B_z(z, t)*Derivative(v_\varphi(\varphi, t), \varphi), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(-v_z(z, t)*Derivative(B_\varphi(\varphi, t), \varphi), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(r*B_z(z, t)*Derivative(v_r(r, t), r), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1})) + Integral(-r*v_z(z, t)*Derivative(B_r(r, t), r), (z, z_1, z_2), (\varphi, \varphi_1, \varphi_2), (r, r_1, r_2), (t, t^n, t^{n + 1}))